## Importações

In [ ]:
# Configuração do logging
import logging
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')

In [ ]:
from PyQt5.QtWidgets import QApplication, QMainWindow, QLabel, QComboBox, QPushButton, QFileDialog, QWidget, QVBoxLayout, QHBoxLayout, QSpacerItem, QSizePolicy, QFrame, QDesktopWidget, QScrollArea, QGridLayout, QMessageBox, QSlider
from PyQt5.QtCore import Qt, QRect, QPoint, QTimer, QFileInfo
from PyQt5.QtGui import QPainter, QPen, QImage, QPixmap, QFont, QColor
import cv2
import sys
import os
import numpy as np
from PIL import Image

## Estilo principal

In [ ]:
def setup_fontes(janela):
    janela.label_fonte = QFont("Segoe UI", 20, QFont.Bold)
    janela.combo_fonte = QFont("Segoe UI", 10)
    janela.btn_fonte = QFont("Segoe UI", 12)
    
def setupStyleSheet():
    estilo = """
                QMainWindow {
                    background-color: white;
                }
                QLabel {
                    font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
                    font-size: 12pt;
                    color: black;
                    padding: 10px;
                    margin: 1px;
                }
                QComboBox {
                    font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
                    font-size: 10pt;
                    color: white;
                    background-color: #7f7f7f;
                    border-radius: 5px;
                    padding: 5px;
                    margin: 1px;
                }
                QPushButton {
                    font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
                    font-size: 10pt;
                    color: white;
                    background-color: #7f7f7f;
                    border-radius: 5px;
                    padding: 5px;
                }
                QPushButton:hover {
                    background-color: gray;
                }
            """
    return estilo

## Funções da Janela Principal

In [ ]:
def selecionar_arq(parent, modo, tipo_arquivo):
    options = QFileDialog.Options()
    options |= QFileDialog.DontUseNativeDialog
    file_dialog = QFileDialog()
    file_dialog.setDirectory(os.path.expanduser("C:/Users/walla/documents/"))

    if tipo_arquivo == "Imagem":
        filter = "Imagens (*.png *.jpg *.jpeg)"
    elif tipo_arquivo == "Vídeo":
        filter = "Vídeos (*.mp4 *.avi)"
    else:
        filter = "Todos os Arquivos (*)"

    file_path, _ = file_dialog.getOpenFileName(
        parent, 
        "Selecionar Arquivo", 
        "", 
        filter, 
        options=options
    )

    if file_path:
        file_info = QFileInfo(file_path)
        extension = file_info.suffix().lower()
        if extension in ['png', 'jpg', 'jpeg']:
            tipo_arquivo = "Imagem"
        elif extension in ['mp4', 'avi']:
            tipo_arquivo = "Vídeo"
        else:
            print(f"Formato de arquivo não suportado: {extension}")
            return
        abrir_segunda_janela(parent, file_path, 100, 100, tipo_arquivo)

def avancar(janela):
    modo = janela.modo.currentText()
    tipo_arquivo = janela.tipo_arquivo.currentText()
    selecionar_arq(janela, modo, tipo_arquivo)

def abrir_segunda_janela(janela, file_path, x, y, tipo_arquivo):
    janela.segunda_janela = JanelaSecundaria(file_path, x, y, tipo_arquivo, janela)
    janela.segunda_janela.showMaximized()

def centralizar_tela(janela): 
    qt_rectangle = janela.frameGeometry()
    center_point = QDesktopWidget().availableGeometry().center()
    qt_rectangle.moveCenter(center_point)
    janela.move(qt_rectangle.topLeft())

## Setup widgets da Janela Principal

In [ ]:
def setup_labels_tipo_arquivo(janela, fonte):
    tipo_arquivo_label = QLabel("Tipo de Arquivo", janela)
    tipo_arquivo_label.setFont(fonte)

    # Definir posição e tamanho proporcionais
    x = int(janela.screen_width * 0.1) 
    y = int(janela.screen_height * 0.15)  
    # tipo_arquivo_label.move(200, 150)
    tipo_arquivo_label.move(x, y)
    tipo_arquivo_label.adjustSize()
    return tipo_arquivo_label


def setup_labels_tipo_modo(janela, fonte):
    modo_label = QLabel("Selecione Modo", janela)
    modo_label.setFont(fonte)
    modo_label.setStyleSheet("padding: 10px; margin: 0px;")
    modo_label.adjustSize()
    x = int(janela.screen_width * 0.35) 
    y = int(janela.screen_height * 0.15) 
    #modo_label.move(670, 150)
    modo_label.move(x, y)
    return modo_label

def setup_combobox_tipo_arquivo(janela, fonte):
    tipo_arquivo = QComboBox(janela)
    tipo_arquivo.addItems(["Imagem", "Vídeo"])
    tipo_arquivo.setMinimumWidth(185)
    tipo_arquivo.setMaximumWidth(185)
    tipo_arquivo.setFont(fonte)
    tipo_arquivo.setMinimumContentsLength(10)
    tipo_arquivo.setSizeAdjustPolicy(QComboBox.AdjustToContents)
    x = int(janela.screen_width * 0.1) 
    y = int(janela.screen_height * 0.2)
    #tipo_arquivo.move(200, 200)
    tipo_arquivo.move(x, y)
    return tipo_arquivo
    
def setup_combobox_modo(janela, fonte):
    modo = QComboBox(janela)
    modo.addItems(["Independente", "Cascata"])
    modo.setMinimumWidth(185)
    modo.setMaximumWidth(185)
    modo.setFont(fonte)
    modo.setMinimumContentsLength(10)
    modo.setSizeAdjustPolicy(QComboBox.AdjustToContents)
    x = int(janela.screen_width * 0.35) 
    y = int(janela.screen_height * 0.2) 
    #modo.move(670, 200)
    modo.move(x, y)
    return modo

def setup_btn_avancar(janela, fonte):
    btn_avancar = QPushButton("Avançar", janela)
    btn_avancar.setFont(fonte)
    btn_avancar.resize(120, 40)
    x = int(janela.screen_width * 0.1) 
    y = int(janela.screen_height * 0.5) 
    #btn_avancar.move(200, 600)
    btn_avancar.move(x, y)
    btn_avancar.clicked.connect(lambda: avancar(janela))
    return btn_avancar

def setup_btn_sair(janela, fonte):
    btn_sair = QPushButton("Sair", janela)
    btn_sair.setFont(fonte)
    btn_sair.setStyleSheet("padding: 10px 45px;")
    btn_sair.resize(120, 40)
    x = int(janela.screen_width * 0.35)
    y = int(janela.screen_height * 0.5) 
    #btn_sair.move(670, 600)
    btn_sair.move(x, y)
    btn_sair.clicked.connect(janela.close)
    return btn_sair

## Janela Principal

In [ ]:
class JanelaPrincipal(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Manipulação de Imagens e Vídeos")
        self.setGeometry(100, 100, 1000, 700)
        screen_geometry = QDesktopWidget().availableGeometry()
        self.screen_width = screen_geometry.width()
        self.screen_height = screen_geometry.height()
        self._setup_ui()

    def _setup_ui(self):
        self.setStyleSheet(setupStyleSheet())
        centralizar_tela(self)
        setup_fontes(self)
        self.tipo_arquivo_label = setup_labels_tipo_arquivo(self, self.label_fonte)
        self.modo_label = setup_labels_tipo_modo(self, self.label_fonte)
        self.modo = setup_combobox_modo(self, self.combo_fonte)
        self.tipo_arquivo = setup_combobox_tipo_arquivo(self, self.combo_fonte)
        self.btn_avancar = setup_btn_avancar(self, self.btn_fonte)
        self.btn_sair = setup_btn_sair(self, self.btn_fonte)

## Widgets da janela secundária

In [ ]:
def setup_label_filtro_j2(janela):
    filtro_label = QLabel("Aplicar Filtro", janela)
    filtro_label.setFont(QFont("Segoe UI", 12))
    filtro_label.setStyleSheet("background-color: transparent;")
    x = int(janela.screen_width * 0.015) 
    y = int(janela.screen_height * 0.01)
    filtro_label.move(x, y) 
    filtro_label.adjustSize()  # Ajustar tamanho ao texto
    return filtro_label

def setup_label_filtro_combo_j2(janela):
    filtro_combo = QComboBox(janela)
    filtro_combo.addItems(["blur", "sharpen", "emboss", "laplacian", "canny", "sobel", "cinza", "binario", "nenhum"])
    filtro_combo.setMinimumWidth(300)
    filtro_combo.setMaximumWidth(300)
    x = int(janela.screen_width * 0.015) 
    y = int(janela.screen_height * 0.07)
    filtro_combo.move(x, y) 
    filtro_combo.adjustSize()
    if janela.tipo_arquivo == "Imagem":
        filtro_combo.currentIndexChanged.connect(lambda: aplicar_filtro_selecionado(janela))
    else:
        filtro_combo.currentIndexChanged.connect(lambda: aplicar_filtro_selecionado_video(janela))
    return filtro_combo

def setup_btn_webcam_j2(janela):
    btn_webcam = QPushButton("Webcam", janela)
    btn_webcam.setFixedSize(140, 30)
    x = int(janela.screen_width * 0.5) 
    y = int(janela.screen_height * 0.05)
    btn_webcam.move(x, y) 
    btn_webcam.setVisible(False)
    btn_webcam.clicked.connect(lambda: abrir_webcam(janela))
    return btn_webcam
    

def setup_btn_play_j2(janela): 
    btn_play = QPushButton("Play", janela)
    btn_play.setFixedSize(140, 30)
    x = int(janela.screen_width * 0.3) 
    y = int(janela.screen_height * 0.05)
    btn_play.move(x, y) 
    btn_play.setVisible(False)
    btn_play.clicked.connect(lambda: play_video(janela))
    return btn_play

def setup_btn_pause_j2(janela):
    btn_pause = QPushButton("Pause", janela)
    btn_pause.setFixedSize(140, 30)
    x = int(janela.screen_width * 0.4) 
    y = int(janela.screen_height * 0.05)
    btn_pause.move(x, y)  
    btn_pause.setVisible(False)
    btn_pause.clicked.connect(lambda: pause_video(janela))
    return btn_pause

def setup_combobox_velocidade(janela):
    combo_velocidade = QComboBox(janela)
    janela.combo_velocidade = combo_velocidade
    combo_velocidade.addItems(["0.25x", "0.5x", "1.0x", "2.0x", "3.0x"])
    combo_velocidade.setCurrentText("1.0x")  # Velocidade padrão
    combo_velocidade.setFixedSize(140, 40)
    x = int(janela.screen_width * 0.8) 
    y = int(janela.screen_height * 0.08)
    combo_velocidade.move(x, y) 
    combo_velocidade.currentIndexChanged.connect(lambda: ajustar_velocidade_video(janela))
    combo_velocidade.setVisible(False)
    return combo_velocidade

def setup_btn_avancar_j2(janela):
    btn_avancar = QPushButton("Acelerar", janela)
    btn_avancar.setFixedSize(140, 30)
    x = int(janela.screen_width * 0.8) 
    y = int(janela.screen_height * 0.05)
    btn_avancar.move(x, y)  
    btn_avancar.clicked.connect(lambda: ajustar_velocidade_video(janela))
    btn_avancar.setVisible(False)
    return btn_avancar
    
def setup_btn_inverter_j2(janela):
    btn_inverter = QPushButton("Inverter", janela)
    btn_inverter.setFixedSize(140, 30)
    x = int(janela.screen_width * 0.7) 
    y = int(janela.screen_height * 0.05)
    btn_inverter.move(x, y)  
    btn_inverter.setVisible(False)
    btn_inverter.clicked.connect(lambda: reverso(janela))
    return btn_inverter

def setup_btn_stop_j2(janela):
    btn_stop = QPushButton("Parar", janela)
    btn_stop.setFixedSize(140, 30)
    x = int(janela.screen_width * 0.6) 
    y = int(janela.screen_height * 0.05)
    btn_stop.move(x, y) 
    btn_stop.setVisible(False)
    btn_stop.clicked.connect(lambda: stop_video(janela))
    return btn_stop

def setup_btn_zoom_mais_j2(janela):
    btn_zoom_mais = QPushButton("Zoom+", janela)
    btn_zoom_mais.setFixedSize(140, 30)
    x = int(janela.screen_width * 0.6) 
    y = int(janela.screen_height * 0.9)
    btn_zoom_mais.move(x, y)
    btn_zoom_mais.clicked.connect(lambda: zoom_in(janela))
    return btn_zoom_mais

def setup_btn_zoom_menos_j2(janela):
    btn_zoom_menos = QPushButton("Zoom-", janela)
    btn_zoom_menos.setFixedSize(140, 30)
    x = int(janela.screen_width * 0.7) 
    y = int(janela.screen_height * 0.9)
    btn_zoom_menos.move(x, y)
    btn_zoom_menos.clicked.connect(lambda: zoom_out(janela))
    return btn_zoom_menos
    
def setup_btn_salvar_j2(janela):
    btn_salvar = QPushButton("Salvar", janela)
    btn_salvar.setFixedSize(140, 30)
    x = int(janela.screen_width * 0.5) 
    y = int(janela.screen_height * 0.9)
    btn_salvar.move(x, y)
    if janela.tipo_arquivo == "Imagem":
        btn_salvar.clicked.connect(lambda: salvar_img_recorte_filtrada_ou_original(janela))
    else:
        btn_salvar.clicked.connect(lambda: salvar_video(janela))
    return btn_salvar

def setup_btn_desfazer_j2(janela):
    btn_desfazer = QPushButton("Desfazer Seleção", janela)
    btn_desfazer.setFixedSize(150, 30)
    btn_desfazer.setStyleSheet("padding: 0, 5px;")
    x = int(janela.screen_width * 0.4) 
    y = int(janela.screen_height * 0.9)
    btn_desfazer.move(x, y)
    btn_desfazer.clicked.connect(lambda: aplicar_tamanho_original(janela))
    return btn_desfazer

def setup_btn_voltar_tela_j2(janela):
    btn_voltar_tela = QPushButton("Voltar", janela)
    btn_voltar_tela.setFixedSize(140, 30)
    x = int(janela.screen_width * 0.2) 
    y = int(janela.screen_height * 0.9)
    btn_voltar_tela.move(x, y)  
    btn_voltar_tela.clicked.connect(lambda: voltar(janela))
    return btn_voltar_tela

def setup_btn_sair_j2(janela):
    btn_sair = QPushButton("Sair", janela)
    btn_sair.setFixedSize(140, 30)
    x = int(janela.screen_width * 0.3) 
    y = int(janela.screen_height * 0.9)
    btn_sair.move(x, y)  
    btn_sair.clicked.connect(lambda: sair(janela))
    return btn_sair

def setup_btn_recorte_j2(janela):
    btn_recorte = QPushButton("Recorte", janela)
    btn_recorte.setFixedSize(140, 30)
    x = int(janela.screen_width * 0.8) 
    y = int(janela.screen_height * 0.9)
    btn_recorte.move(x, y)  
    btn_recorte.clicked.connect(lambda: recorte_video(janela))
    return btn_recorte

def setup_scroll_area_j2(janela):
    scroll_area = QScrollArea(janela)
    scroll_area.setWidgetResizable(True)
    scroll_area.setAlignment(Qt.AlignCenter)
    x = int(janela.screen_width * 0.2) 
    y = int(janela.screen_height * 0.15)
    scroll_area.setFixedSize(int(janela.screen_width * 0.7), int(janela.screen_height * 0.65))
    scroll_area.move(x, y)
    return scroll_area

def setup_image_label_j2(janela):
    image_label = ImageLabel(janela)
    image_label.setAlignment(Qt.AlignCenter)
    janela.scroll_area.setWidget(image_label)
    return image_label

class ImageLabel(QLabel):
    def __init__(self, scroll_area, parent=None):
        super().__init__(parent)
        self.scroll_area = scroll_area 
        self.start_point = QPoint()
        self.end_point = QPoint()
        self.is_selecting = False

    def paintEvent(self, event):
        super().paintEvent(event)
        if self.is_selecting:
            painter = QPainter(self)
            painter.setPen(QPen(Qt.blue, 2, Qt.SolidLine)) 
            painter.setBrush(QColor(0, 0, 255, 50))  
            offset = self.scroll_area.pos()  
            start = self.start_point - offset
            end = self.end_point - offset
    
            area = QRect(start, end).normalized()
            painter.drawRect(area)

def setup_slider_j2(janela):
    slider = QSlider(Qt.Horizontal, janela)
    janela.time_label = QLabel(janela)
    janela.slider = slider
    x = int(janela.screen_width * 0.3) 
    y = int(janela.screen_height * 0.85)
    slider.setGeometry(x, y, 1000, 30)  
    slider.setRange(0, 100)
    slider.sliderMoved.connect(lambda position: set_video_position(janela, position))
    return slider
    
def setup_time_label_j2(janela):
    time_label = QLabel(janela)
    x = int(janela.screen_width * 0.2) 
    y = int(janela.screen_height * 0.84)
    time_label.setGeometry(x, y, 200, 40) 
    time_label.setText("00:00:00 / 00:00:00")
    time_label.setStyleSheet("font-family: 'Segoe UI'; font-size: 10pt;")
    return time_label

## Fuções webcam

In [ ]:
def abrir_webcam(janela):
    janela.capturando_webcam = True
    stop_video(janela)
    janela.cap = cv2.VideoCapture(janela.indice_webcam)
    janela.indice_webcam += 1
    if janela.indice_webcam > 2:
        stop_all(janela)
        janela.indice_webcam = 0
    janela.timer.timeout.connect(lambda: atualizar_frame_webcam(janela))
    janela.timer.start(30)

def atualizar_frame_webcam(janela):
    if janela.capturando_webcam:
        ret, frame = janela.cap.read()
        if not ret:
            return
        exibir_frame(janela, frame)
    else:
        janela.cap.release()
        janela.timer.stop()
        cv2.destroyAllWindows()

def parar_webcam(janela):
    janela.capturando_webcam = False
    janela.cap.release()
    janela.timer.stop()
    cv2.destroyAllWindows()

def stop_all(janela):
    if janela.capturando_webcam:
        parar_webcam(janela)
    else:
        self.stop_video()

def exibir_frame(janela, frame):
    rgb_image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    altura, largura, _ = rgb_image.shape
    nova_largura = int(largura * janela.zoom_level)
    nova_altura = int(altura * janela.zoom_level)
    imagem_pillow = Image.fromarray(rgb_image) 
    imagem_zoom = imagem_pillow.resize((nova_largura, nova_altura), Image.LANCZOS)
    if janela.filter_type:
        imagem_zoom = aplicar_filtro_completo(imagem_zoom, janela.filter_type)
    
    frame_processado = np.array(imagem_zoom)
    janela.lista_processada.append(frame_processado)
    pixmap = pil_imagem_qpixmap(imagem_zoom)
    janela.image_label.adjustSize()
    janela.scroll_area.setWidget(janela.image_label)
    janela.image_label.setPixmap(pixmap.scaled(int(janela.screen_width * 0.7), int(janela.screen_height * 0.65), Qt.KeepAspectRatio, Qt.SmoothTransformation))

## Funções da Janela Secundária

In [ ]:
def voltar(janela): 
    janela.close() 
    janela.janela_principal.show()

def sair(janela):
    msg_box = QMessageBox(janela)
    msg_box.setWindowTitle("Confirmação")
    msg_box.setText("Tem certeza de que deseja sair?")
    btn_sim = msg_box.addButton("Sim", QMessageBox.YesRole)
    btn_nao = msg_box.addButton("Não", QMessageBox.NoRole)
    btn_nao.setDefault(True)
    btn_nao.setFocus()
    msg_box.exec()
    if msg_box.clickedButton() == btn_sim:
        janela.janela_principal.close()
        janela.close()

def carregar_arquivo(janela, file_path):
    logging.debug(f"Carregando arquivo do caminho: {file_path}")
    if janela.tipo_arquivo == "Imagem":
        try:
            janela.original_image = Image.open(file_path)
            carregar_imagem(janela)
            logging.debug("Imagem carregada com sucesso.")
            exibir_controles_de_video(janela, False)
        except Exception as e:
            logging.error(f"Falha ao carregar a imagem: {e}")
    elif janela.tipo_arquivo == "Vídeo":
        try:
            janela.video_capture = cv2.VideoCapture(file_path)
            if not janela.video_capture.isOpened():
                raise ValueError("Erro ao abrir o vídeo.")
            janela.timer.timeout.connect(lambda: atualizar_quadro(janela))
            processar_video(janela)
            logging.debug("Vídeo carregado com sucesso.")
            exibir_controles_de_video(janela, True)
        except Exception as e:
            logging.error(f"Falha ao carregar o vídeo: {e}")

def exibir_controles_de_video(janela, visible):
    janela.combo_velocidade.setVisible(visible)
    janela.btn_play.setVisible(visible)
    janela.btn_pause.setVisible(visible)
    janela.btn_avancar.setVisible(visible)
    janela.btn_stop.setVisible(visible)
    janela.btn_inverter.setVisible(visible)
    janela.btn_webcam.setVisible(visible)
    janela.slider.setVisible(visible)
    janela.time_label.setVisible(visible)
    janela.btn_recorte.setVisible(visible)

def zoom_in(janela):
    if janela.zoom_in_count == 0:
        janela.zoom_level = 1.1
    else:
        janela.zoom_level *= 1.1
    janela.zoom_in_count += 1
    logging.debug(f"Zoom in: level {janela.zoom_level}, count {janela.zoom_in_count}")
    if janela.tipo_arquivo == "Imagem":
        atualizar_imagem(janela)
    else:
        atualizar_quadro(janela)

def zoom_out(janela):
    if janela.zoom_in_count > 1:
        janela.zoom_level /= 1.1
        janela.zoom_in_count -= 1
        logging.debug(f"Zoom out: level {janela.zoom_level}, count {janela.zoom_in_count}")
        if janela.tipo_arquivo == "Imagem":
            atualizar_imagem(janela)
        else:
            atualizar_quadro(janela)
    else:
        if janela.tipo_arquivo == "Imagem":
            aplicar_tamanho_original(janela)
        else:
            janela.zoom_level = 1
            janela.zoom_in_count = 0
            atualizar_quadro(janela)

## Região de interesse

In [ ]:
def recortar_imagem(janela):
    imagem_para_recorte = janela.filtered_image if janela.filtered_image is not None else janela.original_image
    if imagem_para_recorte is not None and not janela.image_label.start_point.isNull() and not janela.image_label.end_point.isNull():
        try:
            start_point = janela.image_label.start_point
            end_point = janela.image_label.end_point

            x1_rel = min(start_point.x(), end_point.x())
            y1_rel = min(start_point.y(), end_point.y())
            x2_rel = max(start_point.x(), end_point.x())
            y2_rel = max(start_point.y(), end_point.y())

            label_width = janela.image_label.width()
            label_height = janela.image_label.height()

            img_width = imagem_para_recorte.width
            img_height = imagem_para_recorte.height

            escala_x = img_width / label_width
            escala_y = img_height / label_height

            x1_img = int(x1_rel * escala_x)
            y1_img = int(y1_rel * escala_y)
            x2_img = int(x2_rel * escala_x)
            y2_img = int(y2_rel * escala_y)

            if x1_img < x2_img and y1_img < y2_img:
                x1_img = max(0, x1_img)
                y1_img = max(0, y1_img)
                x2_img = min(img_width, x2_img)
                y2_img = min(img_height, y2_img)

                janela.recorte = imagem_para_recorte.crop((x1_img, y1_img, x2_img, y2_img))

                if janela.recorte.mode != 'RGB':
                    janela.recorte = janela.recorte.convert('RGB')

                pixmap = pil_imagem_qpixmap(janela.recorte)
                janela.image_label.setPixmap(pixmap)
                janela.image_label.adjustSize()
                janela.scroll_area.setWidget(janela.image_label)

                logging.debug(f"Imagem recortada: {x1_img}, {y1_img}, {x2_img}, {y2_img}")
        except Exception as e:
            logging.error(f"Erro ao recortar imagem: {e}")

def recortar_frame(janela, frame):
    imagem_para_recorte = frame
    if imagem_para_recorte is not None and not janela.image_label.start_point.isNull() and not janela.image_label.end_point.isNull():
        try:
            start_point = janela.image_label.start_point
            end_point = janela.image_label.end_point

            x1_rel = min(start_point.x(), end_point.x())
            y1_rel = min(start_point.y(), end_point.y())
            x2_rel = max(start_point.x(), end_point.x())
            y2_rel = max(start_point.y(), end_point.y())

            label_width = janela.image_label.width()
            label_height = janela.image_label.height()

            img_width = imagem_para_recorte.width
            img_height = imagem_para_recorte.height

            escala_x = img_width / label_width
            escala_y = img_height / label_height

            x1_img = int(x1_rel * escala_x)
            y1_img = int(y1_rel * escala_y)
            x2_img = int(x2_rel * escala_x)
            y2_img = int(y2_rel * escala_y)
            
            if x1_img < x2_img and y1_img < y2_img:
                x1_img = max(0, x1_img)
                y1_img = max(0, y1_img)
                x2_img = min(img_width, x2_img)
                y2_img = min(img_height, y2_img)
                janela.recorte = imagem_para_recorte.crop((x1_img, y1_img, x2_img, y2_img))
                if janela.recorte.mode != 'RGB':
                    janela.recorte = janela.recorte.convert('RGB')
                return janela.recorte
        except Exception as e:
            logging.error(f"Erro ao recortar imagem: {e}")

## Manipulação de Imagem

In [ ]:
def atualizar_imagem(janela):
    # Usar a imagem filtrada se existir, caso contrário usar a original
    imagem_para_exibir = janela.filtered_image if janela.filtered_image is not None else janela.original_image
    if imagem_para_exibir:
        largura, altura = imagem_para_exibir.size
        nova_largura = int(largura * janela.zoom_level)
        nova_altura = int(altura * janela.zoom_level)
        imagem_zoom = imagem_para_exibir.resize((nova_largura, nova_altura), Image.LANCZOS)
        pixmap = pil_imagem_qpixmap(imagem_zoom)
        janela.image_label.setPixmap(pixmap)
        janela.image_label.adjustSize()
        janela.scroll_area.setWidget(janela.image_label)
        logging.debug(f"Imagem atualizada com fator de zoom: {janela.zoom_level}")
    else:
        logging.error("Nenhuma imagem disponível para exibição!")      

def pil_imagem_qpixmap(image):
    """Converte uma imagem PIL para QPixmap"""
    image = image.convert("RGBA")
    date = image.tobytes("raw", "RGBA")
    qimage = QImage(date, image.width, image.height, QImage.Format_RGBA8888)
    pixmap = QPixmap.fromImage(qimage)
    return pixmap

def carregar_imagem(janela):
    if janela.original_image:
        pixmap = pil_imagem_qpixmap(janela.original_image)
        janela.image_label.setPixmap(pixmap.scaled(janela.image_label.size(), Qt.KeepAspectRatio, Qt.SmoothTransformation))
        janela.image_label.adjustSize()
        janela.scroll_area.setWidget(janela.image_label)
        logging.debug("Imagem exibida no QLabel.")
    else:
        logging.error("original_image não está definida!")
        
def aplicar_filtro_completo(image, filter_type):
    cv_image = np.array(image)
    logging.debug(f"Aplicando filtro: {filter_type}")

    if filter_type == "blur":
        kernel = np.ones((3, 3), np.float32) / 9
        filtered_image = cv2.filter2D(cv_image, -1, kernel)
    elif filter_type == "sharpen":
        kernel = np.array([[0, -1, 0],
                           [-1, 5, -1],
                           [0, -1, 0]])
        filtered_image = cv2.filter2D(cv_image, -1, kernel)
    elif filter_type == "emboss":
        kernel = np.array([[-2, -1, 0],
                           [-1,  1, 1],
                           [ 0,  1, 2]])
        filtered_image = cv2.filter2D(cv_image, -1, kernel)
    elif filter_type == "laplacian":
        kernel = np.array([[0, 1, 0],
                           [1, -4, 1],
                           [0, 1, 0]])
        filtered_image = cv2.filter2D(cv_image, -1, kernel)
    elif filter_type == "canny":
        gray_image = cv2.cvtColor(cv_image, cv2.COLOR_BGR2GRAY)
        filtered_image = cv2.Canny(gray_image, 100, 200)
    elif filter_type == "sobel":
        gray_image = cv2.cvtColor(cv_image, cv2.COLOR_BGR2GRAY)
        grad_x = cv2.filter2D(gray_image, -1, np.array([[-1, 0, 1],
                                                       [-2, 0, 2],
                                                       [-1, 0, 1]]))
        grad_y = cv2.filter2D(gray_image, -1, np.array([[-1, -2, -1],
                                                       [ 0,  0,  0],
                                                       [ 1,  2,  1]]))
        filtered_image = cv2.addWeighted(grad_x, 0.5, grad_y, 0.5, 0)
    elif filter_type == "binario":
        gray_image = cv2.cvtColor(cv_image, cv2.COLOR_BGR2GRAY)
        kernel = np.array([[-1, -1, -1],
                           [-1,  8, -1],
                           [-1, -1, -1]])
        filtered_image = cv2.filter2D(gray_image, -1, kernel)
        _, filtered_image = cv2.threshold(filtered_image, 127, 255, cv2.THRESH_BINARY)
    elif filter_type == "cinza":
        filtered_image = cv2.cvtColor(cv_image, cv2.COLOR_BGR2GRAY)
    else:
        logging.debug("Filtro padrão aplicado (sem modificações).")
        filtered_image = cv_image
    logging.debug(f"Filtro {filter_type} aplicado.")
    return Image.fromarray(filtered_image)

def aplicar_filtro_selecionado(janela):
    filter_type = janela.filtro_combo.currentText()
    janela.filtered_image = aplicar_filtro_completo(janela.original_image, filter_type)
    janela.zoom_level = 1.0
    janela.zoom_in_count = 0
    atualizar_imagem_filtrada(janela)

def atualizar_imagem_filtrada(janela):
    if janela.filtered_image:
        pixmap = pil_imagem_qpixmap(janela.filtered_image)
        janela.image_label.setPixmap(pixmap.scaled(janela.image_label.size(), Qt.KeepAspectRatio, Qt.SmoothTransformation))
        janela.image_label.adjustSize()
        janela.scroll_area.setWidget(janela.image_label)
        logging.debug("Imagem filtrada atualizada.")

def opencv_imagem_qpixmap(image):
    altura, largura, canal = image.shape
    bytes_por_linha = canal * largura
    qimage = QImage(image.data, largura, altura, bytes_por_linha, QImage.Format_RGB888)
    pixmap = QPixmap.fromImage(qimage)
    return pixmap

def aplicar_tamanho_original(janela):
    if janela.tipo_arquivo == "Imagem":
        if janela.original_image is not None:
            try:
                original_img_np = cv2.cvtColor(np.array(janela.original_image), cv2.COLOR_RGB2BGR)
                pixmap = pil_imagem_qpixmap(janela.original_image)
                janela.image_label.setPixmap(pixmap.scaled(janela.scroll_area.size(), Qt.KeepAspectRatio, Qt.SmoothTransformation))
                janela.image_label.resize(janela.scroll_area.size())
                janela.scroll_area.setWidget(janela.image_label)
                janela.scroll_area.setWidgetResizable(True)
                print("Imagem redefinida para o tamanho original.")
            except Exception as e:
                print(f"Erro ao redefinir para o tamanho original: {e}")
        else:
            print("Nenhuma imagem carregada para redefinir.")
    else:
        janela.recorte_video = False

def salvar_img_recorte_filtrada_ou_original(janela):
    if janela.filtered_image is not None or janela.original_image is not None or janela.recorte is not None:
        if janela.recorte is not None:
            imagem_para_exibir = janela.recorte
        else:
            imagem_para_exibir = janela.filtered_image if janela.filtered_image is not None else janela.original_image
        imagem_para_exibir = np.array(imagem_para_exibir)
        imagem_para_exibir = cv2.cvtColor(imagem_para_exibir, cv2.COLOR_BGR2RGB)
        options = QFileDialog.Options()
        path_salvar, _ = QFileDialog.getSaveFileName(
            None,
            "Salvar Imagem",
            "",
            "Imagens PNG (*.png);;Imagens JPEG (*.jpg);;Todos os Arquivos (*)",
            options=options
        )
        if path_salvar: 
            cv2.imwrite(path_salvar, imagem_para_exibir)
            msg = QMessageBox()
            msg.setIcon(QMessageBox.Information)
            msg.setWindowTitle("Imagem Salva")
            msg.setText(f"A imagem foi salva com sucesso em:\n{path_salvar}")
            msg.setStandardButtons(QMessageBox.Ok)
            msg.exec_()
        else:
            msg = QMessageBox()
            msg.setIcon(QMessageBox.Warning)
            msg.setWindowTitle("Operação Cancelada")
            msg.setText("A operação de salvar a imagem foi cancelada.")
            msg.setStandardButtons(QMessageBox.Ok)
            msg.exec_()
    else:
        msg = QMessageBox()
        msg.setIcon(QMessageBox.Warning)
        msg.setWindowTitle("Erro")
        msg.setText("Nenhuma imagem disponível para salvar.")
        msg.setStandardButtons(QMessageBox.Ok)
        msg.exec_()

## Manipulação de vídeo

In [ ]:
def atualizar_quadro(janela):
    if janela.capturando_webcam == True:
        return
    try:
        if janela.reproduzindo_reverso == True:
            if janela.frame_index < len(janela.lista):
                frame = janela.lista[janela.frame_index]
                janela.frame_index += 1
                atualizar_slider_e_label(janela)
            else:
                janela.timer.stop()
                mostrar_mensagem_fim_video(janela)
                logging.debug("Fim do vídeo reverso.")
                return
        else:
            ret, frame = janela.video_capture.read()
            if not ret:
                janela.timer.stop()
                mostrar_mensagem_fim_video(janela)
                logging.debug("Fim do vídeo ou erro ao ler o quadro.")
                return
        imagem = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        altura, largura, _ = imagem.shape
        nova_largura = int(largura * janela.zoom_level)
        nova_altura = int(altura * janela.zoom_level)
        imagem_pillow = Image.fromarray(imagem)
        imagem_zoom = imagem_pillow.resize((nova_largura, nova_altura), Image.LANCZOS)
        
        if janela.filter_type:
            imagem_zoom = aplicar_filtro_completo(imagem_zoom, janela.filter_type)
            
        if janela.recorte_video == True:
            imagem_zoom = recortar_frame(janela, imagem_zoom)
            
        frame_processado = np.array(imagem_zoom)
        janela.lista_processada.append(frame_processado)
        pixmap = pil_imagem_qpixmap(imagem_zoom)
        janela.image_label.setPixmap(pixmap)
        janela.image_label.adjustSize()
        janela.scroll_area.setWidget(janela.image_label)
        atualizar_slider_e_label(janela)
        salvar_frame(janela, imagem)
    except Exception as e:
            print(f"Erro ao atualizar quadro: {e}")
        
def format_time(seconds):
    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60)
    seconds = int(seconds % 60)
    return f"{hours:02d}:{minutes:02d}:{seconds:02d}"

def processar_video(janela):
    janela.timer.timeout.connect(lambda: atualizar_quadro(janela))
    janela.timer.start(30) 
    logging.debug("Iniciando processamento do vídeo.")

def set_video_position(janela, position):
    total_frames = int(janela.video_capture.get(cv2.CAP_PROP_FRAME_COUNT))
    new_frame = int((position / 100) * total_frames)
    janela.video_capture.set(cv2.CAP_PROP_POS_FRAMES, new_frame)
    atualizar_quadro(janela)

def play_video(janela):
    if not janela.timer.isActive():
        if janela.reproduzindo_reverso: 
            janela.timer.timeout.connect(lambda: atualizar_quadro(janela))
        else:
            janela.timer.timeout.connect(lambda: atualizar_quadro(janela))
        janela.timer.start(30)
        logging.debug("Reprodução do vídeo iniciada.")

def pause_video(janela):
    if janela.timer.isActive():
        janela.timer.stop()
        logging.debug("Reprodução do vídeo pausada.")

def stop_video(janela):
    janela.timer.stop()
    janela.lista_processada = []
    janela.lista_pos_corte_video = []
    if janela.reproduzindo_reverso:
        janela.frame_index = 0
    else:
        janela.video_capture.set(cv2.CAP_PROP_POS_FRAMES, 0)
    atualizar_quadro(janela)
    logging.debug("Reprodução do vídeo parada.")

def ajustar_velocidade_video(janela):
    velocidade_text = janela.combo_velocidade.currentText()
    velocidade = float(velocidade_text[:-1])
    novo_intervalo = int(30 / velocidade)
    janela.timer.setInterval(novo_intervalo) 
    logging.debug(f"Velocidade do vídeo ajustada para: {velocidade_text} (Intervalo: {novo_intervalo}ms)")

def aplicar_filtro_selecionado_video(janela):
    janela.filter_type = janela.filtro_combo.currentText()
    if janela.capturando_webcam == False:
        atualizar_quadro(janela)
    logging.debug(f"Filtro: {janela.filter_type} aplicado")

def recorte_video(janela):
    if janela.reproduzindo_reverso:
        index = int(janela.frame_index)
        janela.lista_pos_corte_video.append(index)
    else:
        index = int(janela.video_capture.get(cv2.CAP_PROP_POS_FRAMES))
        janela.lista_pos_corte_video.append(index)

## Pos-tratamento de frames

In [ ]:
def mostrar_mensagem_fim_video(janela):
    frame_count = len(os.listdir(janela.output_dir))
    mensagem = QMessageBox(janela)
    mensagem.setWindowTitle("Vídeo Finalizado")
    mensagem.setText(f"O vídeo finalizou! Foram gerados {frame_count} frames.\nVocê deseja salvar em uma pasta local?")
    mensagem.setStandardButtons(QMessageBox.Yes | QMessageBox.No)
    resposta = mensagem.exec_()
    if resposta == QMessageBox.Yes:
        salvar_frames_em_pasta(janela)
    else:
        limpar_frames_temporarios(janela)
        
def salvar_frame(janela, frame):
    janela.output_dir = "frames_temp"
    if not os.path.exists(janela.output_dir):
        os.makedirs(janela.output_dir)
    frame_number = int(janela.video_capture.get(cv2.CAP_PROP_POS_FRAMES))
    frame_filename = os.path.join(janela.output_dir, f"frame_{frame_number:04d}.png")
    cv2.imwrite(frame_filename, cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))
    #logging.debug(f"Frame salvo: {frame_filename}")

def salvar_frames_em_pasta(janela):
    pasta_destino = QFileDialog.getExistingDirectory(janela, "Selecione a Pasta de Destino", "C:/")
    if pasta_destino:
        for filename in os.listdir(janela.output_dir):
            src_path = os.path.join(janela.output_dir, filename)
            dst_path = os.path.join(pasta_destino, filename)
            os.rename(src_path, dst_path)
        logging.debug(f"Frames salvos na pasta: {pasta_destino}")
    janela.limpar_frames_temporarios()

def limpar_frames_temporarios(janela):
    for filename in os.listdir(janela.output_dir):
        os.remove(os.path.join(janela.output_dir, filename))
    os.rmdir(janela.output_dir)
    logging.debug("Frames temporários limpos.")

## Salvar video

In [ ]:
def ajustar_frames(lista_frames, largura, altura):
    frames_ajustados = []
    for frame in lista_frames:
        frame_redimensionado = cv2.resize(frame, (largura, altura), interpolation=cv2.INTER_LINEAR)
        frames_ajustados.append(frame_redimensionado)
    return frames_ajustados

def salvar_video(janela):
    if janela.lista_pos_corte_video:
        salvar_video_em_cortes(janela)
    else:
        salvar_video_completo(janela)
        
def salvar_video_completo(janela):
    if janela.capturando_webcam == True:
        parar_webcam(janela)
    options = QFileDialog.Options()
    path_salvar, _ = QFileDialog.getSaveFileName(
        None,
        "Salvar Vídeo",
        "",
        "Arquivos de Vídeo (*.avi *.mp4);;Todos os Arquivos (*)",
        options=options
    )
    if not path_salvar:
        print("Operação de salvar cancelada.")
        return
        
    if not hasattr(janela, "lista_processada") or len(janela.lista_processada) == 0:
        print("Nenhum frame processado encontrado para salvar o vídeo.")
        return
        
    altura, largura, _ = janela.lista_processada[0].shape
    frames_ajustados = ajustar_frames(janela.lista_processada, largura, altura)
    fps = 1000 // janela.timer.interval()

    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    video_writer = cv2.VideoWriter(path_salvar, fourcc, fps, (largura, altura))

    for frame in frames_ajustados:
        video_writer.write(cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))
    video_writer.release()

    print(f"Vídeo salvo como: {path_salvar}")

def salvar_video_em_cortes(janela):
    options = QFileDialog.Options()
    path_salvar, _ = QFileDialog.getSaveFileName(
        None,
        "Salvar Vídeo",
        "",
        "Arquivos de Vídeo (*.avi *.mp4);;Todos os Arquivos (*)",
        options=options
    )
    if not path_salvar:
        print("Operação de salvar cancelada.")
        return

    if not hasattr(janela, "lista_processada") or len(janela.lista_processada) == 0:
        print("Nenhum frame processado encontrado para salvar o vídeo.")
        return
         
    try:
        altura, largura, _ = janela.lista_processada[0].shape
    except ValueError:
        print("Erro: Os frames processados não têm o formato esperado (altura, largura, canais).")
        return
        
    fps = 1000 // janela.timer.interval()
    pos_cortes = janela.lista_pos_corte_video

    if len(pos_cortes) == 0 or pos_cortes[-1] != len(janela.lista_processada) - 1:
        pos_cortes.append(len(janela.lista_processada) - 1)
        
    segmentos = []
    inicio = 0

    for corte in pos_cortes:
        segmentos.append(janela.lista_processada[inicio:corte + 1])
        inicio = corte + 1
        
    for i, segmento in enumerate(segmentos):
        if len(segmento) == 0:
            continue
            
        frames_ajustados = ajustar_frames(segmento, largura, altura)
        nome_arquivo = f"{path_salvar.rsplit('.', 1)[0]}_corte_{i + 1}.avi"
        fourcc = cv2.VideoWriter_fourcc(*'XVID')
        
        video_writer = cv2.VideoWriter(nome_arquivo, fourcc, fps, (largura, altura))
        for frame in frames_ajustados:
            video_writer.write(cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))
        video_writer.release()
        
        print(f"Vídeo do corte {i + 1} salvo como: {nome_arquivo}")
    print("Todos os cortes foram salvos.")

## Slider e label

In [ ]:
def atualizar_slider_e_label(janela):
    if not janela.capturando_webcam: 
        if janela.reproduzindo_reverso:
            total_frames = len(janela.lista)
            current_frame = janela.frame_index
            fps = janela.video_capture.get(cv2.CAP_PROP_FPS) if janela.video_capture else 30
            current_time = current_frame / fps if fps > 0 else 0
            total_time = total_frames / fps if fps > 0 else 0
            if total_frames > 0:
                position = int((current_frame / total_frames) * 100)
                janela.slider.setValue(position)
            janela.time_label.setText(f"{format_time(current_time)} / {format_time(total_time)}")
        else: 
            if janela.video_capture.isOpened():
                current_frame = int(janela.video_capture.get(cv2.CAP_PROP_POS_FRAMES))
                total_frames = int(janela.video_capture.get(cv2.CAP_PROP_FRAME_COUNT))
                fps = janela.video_capture.get(cv2.CAP_PROP_FPS)
                current_time = current_frame / fps if fps > 0 else 0
                total_time = total_frames / fps if fps > 0 else 0
                if total_frames > 0:
                    position = int((current_frame / total_frames) * 100)
                    janela.slider.setValue(position)
                janela.time_label.setText(f"{format_time(current_time)} / {format_time(total_time)}")

## Reverso

In [ ]:
def reverso(janela):
    janela.lista = []
    cap = cv2.VideoCapture(janela.file_path)
    while True:
        rval, frame = cap.read()
        if not rval:
            break
        janela.lista.append(frame)
    cap.release() 
    janela.lista.reverse() 
    if janela.reproduzindo_reverso:
        janela.reproduzindo_reverso = False
    else:
        janela.reproduzindo_reverso = True
    janela.frame_index = 0 
    processar_video(janela)

## Janela Secundária

In [ ]:
class JanelaSecundaria(QMainWindow):
    def __init__(self, file_path, x, y, tipo_arquivo, parent):
        super().__init__(parent)
        self.setWindowTitle("Manipulação de Imagens e Vídeos")
        self.setGeometry(x, y, 1200, 800)
        screen_geometry = QDesktopWidget().availableGeometry()
        self.screen_width = screen_geometry.width()
        self.screen_height = screen_geometry.height()
        self.file_path = file_path
        self.tipo_arquivo = tipo_arquivo
        self.janela_principal = parent
        self.original_image = None
        self.scaled_image = None  # Nova variável para armazenar a imagem escalada
        self.filtered_image = None # Nova variável para armazenar a imagem filtrada
        self.display_image = None # Variável para armazenar a imagem exibida (com zoom aplicado)
        self.zoom_level = 1.0 # Fator de zoom inicial
        self.initial_zoom_level = self.zoom_level
        self.zoom_in_count = 0 # Contador de cliques no botão Zoom+
        self.start_point = QPoint()# Ponto inicial da seleção de recorte
        self.end_point = QPoint() # Ponto final da seleção de recorte
        self.is_selecting = False # Flag para indicar se o recorte está sendo selecionado
        self.timer = QTimer()
        self.capturando_webcam = False
        self._setup_ui()
        self.showMaximized()
        self.filter_type = None
        self.indice_webcam = 0
        self.reproduzindo_reverso = False
        self.recorte = None
        self.lista_pos_corte_video = []
        self.lista_processada = []
        self.recorte_video = False
        carregar_arquivo(self, file_path)
        
    def _setup_ui(self):
        central_widget = QWidget(self)
        self.setCentralWidget(central_widget)
        self.setStyleSheet(setupStyleSheet())
        self.filtro_label = setup_label_filtro_j2(self)
        self.filtro_combo = setup_label_filtro_combo_j2(self)
        self.btn_play = setup_btn_play_j2(self)
        self.btn_pause = setup_btn_pause_j2(self)
        self.btn_avancar = setup_btn_avancar_j2(self)
        self.btn_inverter = setup_btn_inverter_j2(self)
        self.btn_stop = setup_btn_stop_j2(self)
        self.btn_zoom_mais = setup_btn_zoom_mais_j2(self)
        self.btn_zoom_menos = setup_btn_zoom_menos_j2(self)
        self.btn_salvar = setup_btn_salvar_j2(self)
        self.btn_desfazer = setup_btn_desfazer_j2(self)
        self.btn_voltar_tela = setup_btn_voltar_tela_j2(self)
        self.btn_sair = setup_btn_sair_j2(self)
        self.btn_recorte = setup_btn_recorte_j2(self)
        self.scroll_area = setup_scroll_area_j2(self)
        self.image_label = setup_image_label_j2(self)
        self.btn_webcam = setup_btn_webcam_j2(self)
        self.slider = setup_slider_j2(self)
        self.time_label = setup_time_label_j2(self)
        self.combo_velocidade = setup_combobox_velocidade(self)
        self.slider.setVisible(False) 
        self.time_label.setVisible(False)

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            if self.image_label.geometry().contains(event.pos()):
                self.image_label.start_point = self.image_label.mapFromParent(event.pos())
                self.image_label.end_point = self.image_label.start_point
                self.image_label.is_selecting = True
                self.image_label.update()
    
    def mouseMoveEvent(self, event):
        if self.image_label.is_selecting:
            self.image_label.end_point = self.image_label.mapFromParent(event.pos())
            self.image_label.update()
    
    def mouseReleaseEvent(self, event):
        if event.button() == Qt.LeftButton and self.image_label.is_selecting:
            self.image_label.end_point = self.image_label.mapFromParent(event.pos())
            self.image_label.is_selecting = False
            self.image_label.update()
            if  self.tipo_arquivo == "Imagem":
                recortar_imagem(self)
            else:
                self.recorte_video = True
            logging.debug(f"Seleção finalizada no ponto: {self.image_label.end_point}")    

## Função principal

In [ ]:
def main():
    if __name__ == "__main__":
        app = QApplication(sys.argv)
        janela_principal = JanelaPrincipal()
        janela_principal.show()
        app.exec_()

## Inicio da aplicação

In [ ]:
main()